# NLP Assignment 4

- Name: Anagha Sarmalkar
- 800#: 801077504

### Import the necessary libraries

In [1]:
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import boolean
import re
from functools import reduce
from nltk.stem.porter import *
import math

### Initialze porter stemmer, stopwords and translator to preprocess the corpus

In [2]:
ps = PorterStemmer() 
stop_words = set(stopwords.words('english')) 
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

### Preprocess the corpus

- Clean the text for every document by removing punctuations, stopwords and stemming the words.
- Tokenize the words and store them in a list referencing the document number in a dictionary.
- Sort the dictionary in the ascending order of document ids.

In [3]:
def preprocess(corpus_path):
    corpus={}
    with open("tweets_corpus.txt") as f:
        for line in f:
            key, value=line.split("\t")
            value_list=value.translate(translator).lower().split()
            new_list=[]
            for word in value_list:
                if word not in stop_words:
                    new_list.append(ps.stem(word))
            corpus[int(key)]=new_list
    sorted_corpus = dict(sorted(corpus.items()))
    return sorted_corpus

### Load the corpus

In [4]:
corpus_path="tweets_corpus.txt"
sorted_corpus=preprocess(corpus_path)

In [5]:
sorted_corpus

{81499877556760576: ['bandag',
  'paper',
  'cut',
  'cheesecak',
  'dinner',
  'call',
  'night',
  'doin',
  'big',
  'nyc'],
 81500716438523904: ['krispi',
  'kreme',
  'strawberri',
  'trifl',
  'sinc',
  'start',
  'gym',
  'cri'],
 81503002321616896: ['bacon',
  'cheddar',
  'slider',
  'top',
  'w',
  'fri',
  'egg',
  'blue',
  'chees',
  'slider',
  'top',
  'w',
  'avocado',
  'purpl',
  'cheroke',
  'tomato'],
 81507775422791680: ['nacho', 'w', 'chees', 'shirt', 'uggghhh'],
 81534165975171072: ['aint',
  'nuffin',
  'piec',
  'chees',
  'without',
  'corner',
  'word',
  'never',
  'slice',
  'bitch'],
 81535634019323904: ['tag',
  'usernam',
  'tag',
  'usernam',
  'mmmm',
  'chees',
  'dream',
  'squirrel',
  'burger',
  'chees'],
 81577509950459904: ['mmmm', 'chees'],
 81582996083326976: ['tag',
  'usernam',
  '1st',
  'like',
  '1',
  'year',
  'younger',
  'u',
  '2nd',
  'age',
  'number',
  '3rd',
  'ima',
  'cater',
  'ur',
  'wed',
  'wit',
  'patti',
  'n',
  'chee

## 1. Function for Inverted Index

- Every word in the document is referenced by a list of document it appears in.
- It is stored in a dictionary in the ascending order of keys.

In [6]:
def inverted_index(corpus):
    vocabulary={}
    for key,value in corpus.items():
        for word in value:
            if word not in vocabulary:
                vocabulary[word]=[]
            if key in vocabulary[word]:
                pass
            else:
                vocabulary[word].append(key)
    sorted_vocab = dict(sorted(vocabulary.items()))
    return sorted_vocab

In [7]:
sorted_vocab = inverted_index(sorted_corpus)

In [8]:
sorted_vocab

{'1': [81582996083326976],
 '100': [81844590625304576],
 '1st': [81582996083326976],
 '2': [81716618236928000],
 '2nd': [81582996083326976],
 '38': [81716618236928000],
 '3rd': [81582996083326976],
 'age': [81582996083326976],
 'aint': [81534165975171072],
 'also': [81715158593966080],
 'asparagu': [82650970722533376],
 'ass': [81644157432643584],
 'ate': [82650970722533376],
 'avocado': [81503002321616896],
 'bacon': [81503002321616896,
  81736742478155778,
  82650970722533376,
  85032815321825280,
  86441828815089664],
 'banana': [82461950231064576],
 'bandag': [81499877556760576],
 'bear': [81716618236928000],
 'beauti': [82461950231064576],
 'befor': [81600113016971264],
 'berri': [81656304107651072],
 'big': [81499877556760576],
 'biscuit': [82650970722533376],
 'bit': [81736742478155778],
 'bitch': [81534165975171072],
 'blend': [81656304107651072],
 'blue': [81503002321616896],
 'burger': [81535634019323904],
 'c': [81715158593966080],
 'cake': [85094773555335168],
 'call': [814

## 2. Function for Merge Algorithm
- The sample user queries are taken in a list.
- Every element in the list is tokenized and separated according to the boolean operators that are included.
- The query is optimized for every AND operation by adding the terms (set.intersection operation) in the increasing order of frequencies.
- The OR operation is realized by using the set.union operation.

In [9]:
 def process_query(query_list):   
    operands=['or','not']
    brac_op={}
    for user_query in query_list:
        query = user_query.lower()
        main_query = re.split(r"\s+(?=[^()]*(?:\(|$))", query)
        op=[]
        or_op={}
        and_op={}
        for query in main_query:
            if query in operands:
                op.append(query) 
            else:
                query = query.split('or')
                for sub_query in query: #subqueries to be or-ed
                    sub_query1 = sub_query.split('and')
                    for term in sub_query1: #terms to be and-ed
                        term=term.translate(translator).strip()
                        search = ps.stem(term)
                        if search in sorted_vocab.keys():
                            and_op[search] = sorted_vocab[search]
                        else:
                            and_op[search]=[]
                    and_op = dict(sorted(and_op.items(), key=lambda kv: (len(kv[1]), kv[0])))
                    if 'and' in sub_query:
                        ans = reduce(set.intersection, (set(val) for val in and_op.values()))
                        or_op[sub_query]=ans
                        brac= reduce(set.union, (set(val) for val in or_op.values()))
                    else:
                        brac= reduce(set.union, (set(val) for val in and_op.values()))
        brac_op[user_query]=brac
    return brac_op

### Sample queries and their results

In [10]:
query_list=['(egg AND cheese)',
            '(chocolate AND strawberry)',
            '(eggs AND cheese AND bacon)',
            '(chocolate OR strawberry)',
            '((eggs AND cheese) OR (eggs AND bacon))']

retrieved_docs=process_query(query_list)

In [11]:
retrieved_docs

{'(egg AND cheese)': {81503002321616896,
  81587643376336896,
  81673244926685184,
  81736742478155778,
  82650970722533376,
  85032815321825280,
  86441828815089664},
 '(chocolate AND strawberry)': {81656304107651072, 85094773555335168},
 '(eggs AND cheese AND bacon)': {81503002321616896,
  81736742478155778,
  82650970722533376,
  85032815321825280,
  86441828815089664},
 '(chocolate OR strawberry)': {81500716438523904,
  81623945064890368,
  81656304107651072,
  81715158593966080,
  81716618236928000,
  81842384404623360,
  81844590625304576,
  82461950231064576,
  85032815321825280,
  85094773555335168},
 '((eggs AND cheese) OR (eggs AND bacon))': {81503002321616896,
  81587643376336896,
  81673244926685184,
  81736742478155778,
  82650970722533376,
  85032815321825280,
  86441828815089664}}

## 3. Function for TF-IDF Scoring

- Given a query q and a document d, the TF-IDF score of the document has been calculated by summing tf-idf for every term t in q.
- TF-IDF displays the importance of the query terms in the document.
- TF-IDF is 0 for the terms not present in the document.

In [12]:
def tf_idf_scoring(query_list):
    operands=['or','and']
    query_terms=[]
    doc_score={}
    query_doc={}
    query_sample=query_list[0]
    query = query_sample.lower().translate(translator).split()
    for term in query:
        if term not in operands:
            query_terms.append(ps.stem(term))
    # for value in sample[query_sample]:
    for value in sorted_corpus.keys():
        term_doc_freq=len(sorted_corpus[value])
        tf_idf=0
        for term in query_terms:
            term_count=sorted_corpus[value].count(term)
            tf = (term_count/term_doc_freq)
            idf = math.log(len(sorted_corpus)/len(sorted_vocab[term]))
            tf_idf=tf_idf+(tf*idf)
        doc_score[value]=["Score:", tf_idf]
    query_doc[query_sample]=doc_score
#     print(query_doc)
    return query_doc

In [13]:
sample_query=['(egg AND cheese)']
tfidf = tf_idf_scoring(sample_query)

In [14]:
tfidf

{'(egg AND cheese)': {81499877556760576: ['Score:', 0.0],
  81500716438523904: ['Score:', 0.0],
  81503002321616896: ['Score:', 0.11532800963619008],
  81507775422791680: ['Score:', 0.12262089457728179],
  81534165975171072: ['Score:', 0.06131044728864089],
  81535634019323904: ['Score:', 0.12262089457728179],
  81577509950459904: ['Score:', 0.30655223644320445],
  81582996083326976: ['Score:', 0.030655223644320446],
  81587643376336896: ['Score:', 0.20502757268656013],
  81600113016971264: ['Score:', 0.0],
  81623945064890368: ['Score:', 0.0],
  81644157432643584: ['Score:', 0.0],
  81656304107651072: ['Score:', 0.0],
  81673244926685184: ['Score:', 0.16774983219809467],
  81715158593966080: ['Score:', 0.0],
  81716618236928000: ['Score:', 0.0],
  81736742478155778: ['Score:', 0.30754135902984014],
  81842384404623360: ['Score:', 0.0],
  81844590625304576: ['Score:', 0.0],
  82461950231064576: ['Score:', 0.0],
  82650970722533376: ['Score:', 0.11532800963619008],
  85032815321825280: 

### Document 81736742478155778 has the highest TF-IDF score for the query (egg AND cheese)